## Challenge Problem 1 (Moderate): Log File Analyzer

Problem:
You're given a log file where each line contains a timestamp, log level, and message in this format:

- 2024-10-15 14:23:01 ERROR Database connection failed
- 2024-10-15 14:23:45 INFO User logged in successfully
- 2024-10-15 14:24:12 WARNING Memory usage at 85%
- 2024-10-15 14:25:03 ERROR File not found: data.csv
- 2024-10-15 14:26:17 INFO Processing completed

Write a function analyze_logs(filepath) that:

- Reads the log file
- Returns a dictionary with log level statistics:
    - Count of each log level
    - List of all unique messages for each log level
    - Most recent timestamp for each log level

In [ ]:
# Example Output:

{
    'ERROR': {
        'count': 2,
        'messages': ['Database connection failed', 'File not found: data.csv'],
        'last_seen': '2024-10-15 14:25:03'
    },
    'INFO': {
        'count': 2,
        'messages': ['User logged in successfully', 'Processing completed'],
        'last_seen': '2024-10-15 14:26:17'
    },
    'WARNING': {
        'count': 1,
        'messages': ['Memory usage at 85%'],
        'last_seen': '2024-10-15 14:24:12'
    }
}

### Requirements:

- Use with statement for file handling
- Specify UTF-8 encoding
- Handle empty files gracefully (return empty dict)
- Messages should be stored without duplicates (if same message appears twice, only store once)
- Timestamps should be stored as strings (no datetime parsing needed)

Test File Creation:
Create a test file called test_logs.txt with the sample data above to test your solution.

### How to verify:

- Check that counts are correct for each level
- Verify messages list has no duplicates
- Confirm last_seen shows the most recent timestamp for each level
- Test with an empty file

### Input file has 10 entries total:

- 4 ERRORs (but only 3 unique messages)
- 4 INFOs (but only 3 unique messages)
- 2 WARNINGs (2 unique messages)

#### Solution:

In [ ]:
def analyze_logs(file):

    log_stats = {}

    # Get each line from the file
    # Get a count of each occurence of the keys in each line


    with open(file, encoding="utf-8") as f:
        lines = [x.split() for x in f]

        for line in lines:
            timestamp = line[0] + line[1]
            key = line[2]
            message = line[3:]

            

    # For testing:
    # print(line)
    # print(timestamp)
    # print(key)
    # print(message)    

    return log_stats

In [7]:
analyze_logs("test_logs.txt")

[['2024-10-15', '14:23:01', 'ERROR', 'Database', 'connection', 'failed'], ['2024-10-15', '14:23:45', 'INFO', 'User', 'logged', 'in', 'successfully'], ['2024-10-15', '14:24:12', 'WARNING', 'Memory', 'usage', 'at', '85%'], ['2024-10-15', '14:25:03', 'ERROR', 'File', 'not', 'found:', 'data.csv'], ['2024-10-15', '14:26:17', 'INFO', 'Processing', 'completed'], ['2024-10-15', '14:27:33', 'ERROR', 'Database', 'connection', 'failed'], ['2024-10-15', '14:28:09', 'INFO', 'User', 'logged', 'in', 'successfully'], ['2024-10-15', '14:29:45', 'WARNING', 'Disk', 'space', 'low'], ['2024-10-15', '14:30:12', 'ERROR', 'Network', 'timeout'], ['2024-10-15', '14:31:01', 'INFO', 'Backup', 'started']]
['2024-10-1514:23:01', '2024-10-1514:23:45', '2024-10-1514:24:12', '2024-10-1514:25:03', '2024-10-1514:26:17', '2024-10-1514:27:33', '2024-10-1514:28:09', '2024-10-1514:29:45', '2024-10-1514:30:12', '2024-10-1514:31:01']
['ERROR', 'INFO', 'WARNING', 'ERROR', 'INFO', 'ERROR', 'INFO', 'WARNING', 'ERROR', 'INFO']
[[

{'ERROR': {}, 'INFO': {}, 'WARNING': {}}